# Importing the nexessary libraries

In [1]:
import collections
import numpy as np
import json
import tensorflow.strings as tf_strings
from keras.preprocessing.text import tokenizer_from_json

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Dense, LSTM, Dropout, Activation, TimeDistributed, TextVectorization
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

# Verify Access to GPU

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6899519954572762795
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2236245607
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16862718671271133121
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [3]:
## loading the tokenizers

model_fr = load_model('english_to_french_lstm_model')
model_hi = load_model('english_to_hindi_lstm_model')

#load Tokenizer
with open('english_tokenizer.json') as f:
    data = json.load(f)
    english_tokenizer = tokenizer_from_json(data)

with open('english_tokenizer_hindi.json') as f:
    data = json.load(f)
    english_tokenizer = tokenizer_from_json(data)
    
with open('french_tokenizer.json') as f:
    data = json.load(f)
    french_tokenizer = tokenizer_from_json(data)

with open('hindi_tokenizer.json') as f:
    data = json.load(f)
    hindi_tokenizer = tokenizer_from_json(data)

In [4]:
with open('sequence_length.json') as f:
    max_length = json.load(f)
    
def pad(x, length=None):
    return pad_sequences(x, maxlen=length, padding='post')

def translate_to_french(english_sentence):
    english_sentence = english_sentence.lower()
    
    english_sentence = english_sentence.replace(".", '')
    english_sentence = english_sentence.replace("?", '')
    english_sentence = english_sentence.replace("!", '')
    english_sentence = english_sentence.replace(",", '')
    
    english_sentence = english_tokenizer.texts_to_sequences([english_sentence])
    english_sentence = pad(english_sentence, max_length)
    
    english_sentence = english_sentence.reshape((-1,max_length))
    
    french_sentence = model_fr.predict(english_sentence)[0]
    
    french_sentence = [np.argmax(word) for word in french_sentence]

    french_sentence = french_tokenizer.sequences_to_texts([french_sentence])[0]
    
    print("French translation: ", french_sentence)
    
    return french_sentence

In [5]:
with open('sequence_length_hindi.json') as f:
    max_length_hindi = json.load(f)
    
def pad(x, length=None):
    return pad_sequences(x, maxlen=length, padding='post')

def translate_to_hindi(english_sentence):
    english_sentence = english_sentence.lower()
    
    english_sentence = english_sentence.replace(".", '')
    english_sentence = english_sentence.replace("?", '')
    english_sentence = english_sentence.replace("!", '')
    english_sentence = english_sentence.replace(",", '')
    
    english_sentence = english_tokenizer.texts_to_sequences([english_sentence])
    english_sentence = pad(english_sentence, max_length_hindi)
    
    english_sentence = english_sentence.reshape((-1,max_length_hindi))
    
    hindi_sentence = model_hi.predict(english_sentence)[0]
    
    hindi_sentence = [np.argmax(word) for word in hindi_sentence]

    hindi_sentence = hindi_tokenizer.sequences_to_texts([hindi_sentence])[0]
    
    print("hindi translation: ", hindi_sentence)
    
    return hindi_sentence

# Dual Translation

In [7]:
english_text = input("Enter english sentence: ")
if len(english_text) != 10:
    print("Input sentence must be exactly 10 letters.")
else:
    translate_to_french(english_text)
    translate_to_hindi(english_text)

1/1 [==============================] - 5s 5s/step
French translation:  elle est est est
1/1 [==============================] - 1s 778ms/step
hindi translation:  वह के है।
